### Import Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from modules.utils import load_json, save_json
from tqdm import tqdm  
import os 
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

In [5]:
def get_view_column_splits(data: pd.DataFrame) -> dict:
    """
    Dynamically split columns into groups of 200.
    Adapts automatically for datasets with 201, 401, or 601 columns.
    """
    view_column_splits = {}
    step = 200  # size per chunk

    # Start from column index 1 to skip 'Sample ID' or first column
    total_cols = data.shape[1] - 1
    num_groups = (total_cols + step - 1) // step  # ceiling division

    for i in range(num_groups):
        start = 1 + i * step
        end = min(1 + (i + 1) * step, data.shape[1])
        view_column_splits[i + 1] = data.columns[start:end]

    return view_column_splits 

def run_pipeline(dataset_name: str, experiment_name: str, view_column_splits: dict = None):
    """
    Pipeline that:
      - Loads baseline JSON results and converts to a DataFrame
      - Loads post-feature-selection cv results CSV
      - Concatenates baseline + fs results, shortens featureSelector names, saves combined CSV
      - Copies ranks and validation CSVs into results folder, normalizes names, augments validation with ML metrics
      - Saves val_results and small JSON view mapping
    
    Inputs:
      - dataset_name (str)
      - experiment_name (str)
      - view_column_splits (dict) optional (if not provided an empty dict will be used)
    """
    view_column_splits = view_column_splits or {}
    base_bench = Path(f"../BENCHMARKING/{dataset_name}/{experiment_name}")
    
    # --- Load Baseline Performance ---
    baseline_path = base_bench / "ML-Baseline.json"
    baseline = load_json(baseline_path)
    
    baseline_results = {
        'featureSelector': [],
        'modelName': [], 
        'numFeatures': [],  
        'MeanAccuracy': [],
        'StdAccuracy': [], 
        'MeanPrecision': [], 
        'StdPrecision':[], 
        'MeanRecall':[], 
        'StdRecall': [], 
        'MeanF1': [], 
        'StdF1': [], 
        'MeanAUC': [],
        'StdAUC':[],
        "MeanSpecificity": [],
        "StdSpecificity": [], 
        "MeanNPV": [],
        "StdNPV": [],
        "MeanLR_PLUS": [], 
        "StdLR_PLUS": [],
        "MeanLR_MINUS": [],
        "StdLR_MINUS": []
    }
    
    if baseline:
        for modelname, model_data in baseline.items(): 
            baseline_results['featureSelector'].append('NONE') 
            baseline_results['modelName'].append(modelname)
            baseline_results['numFeatures'].append(600)
            
            if modelname in ['MORE', 'MOGONET']:
                src = model_data  # top-level keys for these models
            else:
                src = model_data.get('cross_val_report', {})  # nested for other models
            
            # now read everything from src (safe .get with defaults)
            baseline_results['MeanAccuracy'].append(src.get('accuracy', {}).get('mean'))
            baseline_results['StdAccuracy'].append(src.get('accuracy', {}).get('std'))
            baseline_results['MeanPrecision'].append(src.get('precision', {}).get('mean'))
            baseline_results['StdPrecision'].append(src.get('precision', {}).get('std'))
            baseline_results['MeanRecall'].append(src.get('recall', {}).get('mean'))
            baseline_results['StdRecall'].append(src.get('recall', {}).get('std'))
            baseline_results['MeanF1'].append(src.get('f1', {}).get('mean'))
            baseline_results['StdF1'].append(src.get('f1', {}).get('std'))
            baseline_results['MeanAUC'].append(src.get('roc_auc', {}).get('mean'))
            baseline_results['StdAUC'].append(src.get('roc_auc', {}).get('std'))
            baseline_results['MeanSpecificity'].append(src.get('specificity', {}).get('mean'))
            baseline_results['StdSpecificity'].append(src.get('specificity', {}).get('std'))
            baseline_results['MeanNPV'].append(src.get('npv', {}).get('mean'))
            baseline_results['StdNPV'].append(src.get('npv', {}).get('std'))
            baseline_results['MeanLR_PLUS'].append(src.get('lr_plus', {}).get('mean'))
            baseline_results['StdLR_PLUS'].append(src.get('lr_plus', {}).get('std'))
            baseline_results['MeanLR_MINUS'].append(src.get('lr_minus', {}).get('mean'))
            baseline_results['StdLR_MINUS'].append(src.get('lr_minus', {}).get('std'))

        baseline_results_df = pd.DataFrame(baseline_results)
    else:
        baseline_results_df = pd.DataFrame(baseline_results)
        print("[run_pipeline] baseline empty -> created empty baseline DataFrame")
    
    # --- Load Performance after feature selection ---
    fs_cv_path = base_bench / "cross-validation-results.csv"
    if fs_cv_path.exists():
        fs_performance = pd.read_csv(fs_cv_path)

    else:
        fs_performance = pd.DataFrame()
        print(f"[run_pipeline] WARNING: {fs_cv_path} not found. Using empty DataFrame for fs_performance.")
    
    # --- combine performance ---
    cv_performance = pd.concat([baseline_results_df, fs_performance], axis=0, ignore_index=True, sort=False)
    
    # Shorten long names
    map_long_names = {
        'randomforest_feature_importance': 'RF-FI',
        'xgb_feature_importance': 'XGB-FI',
        'rf_permutation_feature_importance': 'RF-PFI',
        'xgb_permutation_feature_importance': 'XGB-PFI'
    }
    if 'featureSelector' in cv_performance.columns:
        cv_performance['featureSelector'] = cv_performance['featureSelector'].apply(lambda x: map_long_names[x] if x in map_long_names else x)
    
    # --- copy Ranks ---
    ranks_in = base_bench / "BiomarkerRanks.csv"

    if ranks_in.exists():
        ranks = pd.read_csv(ranks_in)
    else:
        print(f"[run_pipeline] WARNING: ranks file not found at {ranks_in}.")
    
    # --- validation results ---
    val_in = base_bench / "Biomarker-validation-results.csv"
    if val_in.exists():
        val_results = pd.read_csv(val_in)
        # normalize names and columns
        val_results = val_results.rename(columns={"Method": "featureSelector"}) if "Method" in val_results.columns else val_results
        if 'featureSelector' in val_results.columns:
            val_results['featureSelector'] = val_results['featureSelector'].apply(lambda x: map_long_names[x] if x in map_long_names else x)
    else:
        val_results = pd.DataFrame()
    
    # --- Add ML Performance to validation results (full_val_results) ---
    full_val_results = pd.DataFrame()
    if not cv_performance.empty and not val_results.empty:
        # columns to merge from cv_performance (drop featureSelector and numFeatures)
        cv_columns = [c for c in cv_performance.columns if c not in ("featureSelector", "numFeatures")]
        
        # Ensure numeric numFeatures in cv_performance
        if 'numFeatures' in cv_performance.columns:
            # If numFeatures read as float, convert to int where appropriate
            try:
                cv_performance['numFeatures'] = cv_performance['numFeatures'].astype(int)
            except Exception:
                pass
        
        for i in range(val_results.shape[0]):
            selector = val_results.at[i, 'featureSelector'] if 'featureSelector' in val_results.columns else None
            num_features = val_results.at[i, 'method_cutoff'] if 'method_cutoff' in val_results.columns else None
            
            # only process expected cutoffs
            if num_features in range(10, 101, 10) and selector is not None:
                df2 = cv_performance.loc[
                    (cv_performance['featureSelector'] == selector) & 
                    (cv_performance['numFeatures'] == num_features),
                    cv_columns
                ].copy()
                
                if df2.empty:
                    # no matching rows found
                    continue
                
                # rename Mean -> Model (as in original)
                df2 = df2.rename(columns={col: col.replace("Mean", "Model") for col in df2.columns})
                
                # duplicate the validation row to match df2 number of rows
                row = val_results.iloc[i:i+1, :].reset_index(drop=True)
                repeated_row = pd.concat([row]*df2.shape[0], ignore_index=True).reset_index(drop=True)
                
                df_chunk = pd.concat([repeated_row.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
                full_val_results = pd.concat([full_val_results, df_chunk], axis=0, ignore_index=True, sort=False)
        
        # final tidy
        full_val_results = full_val_results.rename(columns={"method_cutoff": "numFeatures"}) if not full_val_results.empty else full_val_results
        # drop fully empty columns/rows
        full_val_results = full_val_results.T.dropna(how='all').T
    else:
        print("[run_pipeline] Skipping ML augmentation: cv_performance or val_results is empty.")
    
    # --- final prints ---
    if 'validationsource' in val_results.columns:
        try:
            counts = val_results['validationsource'].value_counts()
        except Exception as e:
            print(f"[run_pipeline] could not print validationsource counts: {e}")
    else:
        print("[run_pipeline] 'validationsource' column not present in validation results (no counts printed).")
    
    # return a dict of produced DataFrames for convenience if the caller wants them
    return {
        "cv_performance": cv_performance,
        "baseline_results_df": baseline_results_df,
        "ranks": (ranks if 'ranks' in locals() else pd.DataFrame()),
        "val_results": (val_results if 'val_results' in locals() else pd.DataFrame()),
        "full_val_results": (full_val_results if 'full_val_results' in locals() else pd.DataFrame()),
    }


In [6]:
def make_rank_matrix(ranks_df, fill_missing=None, one_based=True):
    """
    Transform a wide rank DataFrame (methods as columns, ranked features as rows)
    into a rank matrix where rows are features, columns are methods, 
    and cells contain the rank of the feature under that method.

    Parameters
    ----------
    ranks_df : pd.DataFrame
        Input DataFrame, each column is a method, each row is a rank position
        containing the feature selected at that rank.
    fill_missing : int or None, optional
        Value to fill in for features not ranked by a method.
        - If None: leaves NaN
        - If int: fills with that value
        - If "max+1": fills with (number of rows + 1), i.e., worst rank
    one_based : bool, default=True
        If True, ranks are 1-based (1, 2, …). If False, ranks are 0-based (0, 1, …).

    Returns
    -------
    rank_matrix : pd.DataFrame
        Features × Methods DataFrame of ranks.
    """
    # Melt into long form with explicit rank positions
    df_long = (
        ranks_df
        .reset_index()
        .rename(columns={'index': 'pos'})
        .melt(id_vars='pos', var_name='featureSelector', value_name='feature')
        .dropna(subset=['feature'])
    )
    
    # Compute rank
    df_long['rank'] = df_long['pos'] + (1 if one_based else 0)

    # Pivot to feature × method
    rank_matrix = df_long.pivot_table(
        index='feature',
        columns='featureSelector',
        values='rank',
        aggfunc='min'
    )

    # Reindex to include all features observed
    all_features = pd.Index(pd.unique(ranks_df.values.ravel())).dropna()
    rank_matrix = rank_matrix.reindex(all_features)

    # Fill missing if requested
    if fill_missing is not None:
        if fill_missing == "max+1":
            fill_value = ranks_df.shape[0] + (1 if one_based else 0)
        else:
            fill_value = fill_missing
        rank_matrix = rank_matrix.fillna(fill_value).astype(int)

    return rank_matrix


### Load and Prepare Results

In [11]:
# set dataset name
experimental_designs = {"ROSMAP":['miRNA_data',
                                  'dna_methylation_data',
                                  'gene_expression_data',
                                  'miRNA_and_gene_expression_data',
                                  'miRNA_and_dna_methylation_data',
                                  'gene_expression_and_dna_methylation_data',
                                  'miRNA_and_gene_expression_and_dna_methylation_data'
                                 ],
                        'MayoRNASeq':[
                            'metabolomics_data',
                            'gene_expression_data',
                            'proteomics_data',
                            'gene_expression_and_proteomics_data',
                            'metabolomics_and_gene_expression_data',
                            'metabolomics_and_proteomics_data',
                            'metabolomics_and_gene_expression_and_proteomics_data'
                            
                        ],
                        'BRCA':['miRNA_data',
                                  'dna_methylation_data',
                                  'gene_expression_data',
                                  'miRNA_and_gene_expression_data',
                                  'miRNA_and_dna_methylation_data',
                                  'gene_expression_and_dna_methylation_data',
                                  'miRNA_and_gene_expression_and_dna_methylation_data'
                                 ]
                       }  

omics_levels_map = {
    'miRNA_data': "SingleOmics",
    'dna_methylation_data': "SingleOmics",
    'gene_expression_data': "SingleOmics",
    'miRNA_and_gene_expression_data': "DualOmics",
    'miRNA_and_dna_methylation_data': "DualOmics",
    'gene_expression_and_dna_methylation_data': "DualOmics",
    'miRNA_and_gene_expression_and_dna_methylation_data': "TripleOmics",
    'metabolomics_data': "SingleOmics",
    'proteomics_data': "SingleOmics",
    'gene_expression_and_proteomics_data': "DualOmics",
    'metabolomics_and_gene_expression_data': "DualOmics",
    'metabolomics_and_proteomics_data': "DualOmics",
    'metabolomics_and_gene_expression_and_proteomics_data': "TripleOmics"
}

omics_types_map = {
    'miRNA_data': "miRNA",
    'dna_methylation_data': "Meth",
    'gene_expression_data': "mRNA",
    
    # Dual-omics
    'miRNA_and_gene_expression_data': "miRNA+mRNA",
    'miRNA_and_dna_methylation_data': "miRNA+Meth",
    'gene_expression_and_dna_methylation_data': "mRNA+Meth",
    'gene_expression_and_proteomics_data': "mRNA+Prot",
    'metabolomics_and_gene_expression_data': "mRNA+Metab",
    'metabolomics_and_proteomics_data': "Metab+Prot",
    
    # Triple-omics
    'miRNA_and_gene_expression_and_dna_methylation_data': "miRNA+mRNA+Meth",
    'metabolomics_and_gene_expression_and_proteomics_data': "mRNA+Metab+Prot",
    
    # Single other omics
    'metabolomics_data': "Metab",
    'proteomics_data': "Prot"
}


Cross_Validation_Results = pd.DataFrame()
Biomarker_Validation_Results = pd.DataFrame()
Cross_Validation_and_Biomarker_Validation_Results = pd.DataFrame()
Selected_Biomarker_Panels = {
    "Feature": [],
    "Cohort": [],
    "OmicsLevel": [],
    "OmicsTypes": []
}

featureSelectors = ['MOGONET:Ranker',
 'MORE:Ranker',
 'boruta',
 'elasticnet',
 'geom.mean_rank',
 'geom.mean_weight',
 'lasso',
 'lime',
 'mannwhitneyu',
 'max_weight',
 'mean_rank',
 'mean_weight',
 'median_rank',
 'median_weight',
 'min_rank',
 'mra_rank',
 'randomforest_feature_importance',
 'rf_permutation_feature_importance',
 'ridge',
 'rra_rank',
 'shap',
 'stuart_rank',
 'svm_rfe',
 't_test',
 'ta_weight',
 'xgb_feature_importance',
 'xgb_permutation_feature_importance']

for featureSelector in featureSelectors:
    Selected_Biomarker_Panels[featureSelector] = []

for dataset_name, experiment_list in experimental_designs.items():
    print(f"\nWORKING ON {dataset_name} DATASET")
    df = pd.read_csv(f'../data/{dataset_name}/prepared/{experiment_list[-1]}.csv', index_col=0) 
    
    for experiment_name in experiment_list:
        print(f"WORKING ON {experiment_name} EXPERIMENT")
        view_column_splits = get_view_column_splits(df) 
        cleaned_results = run_pipeline(dataset_name, experiment_name, view_column_splits) 

        added_attributes = pd.DataFrame({
            "Cohort": [dataset_name],
            "OmicsLevel": [omics_levels_map[experiment_name]],
            "OmicsType": [omics_types_map[experiment_name]],
        })

        # Extend Cross_Validation_Results
        added_attributes_extended = pd.concat([added_attributes] * cleaned_results['cv_performance'].shape[0], ignore_index=True)
        Cross_Validation_Results_current =  pd.concat([added_attributes_extended,  cleaned_results['cv_performance']], axis = 1)
        Cross_Validation_Results = pd.concat([Cross_Validation_Results, Cross_Validation_Results_current], axis = 0)

        # Extend Biomarker_Validation_Results
        if cleaned_results['val_results'].shape[0] > 0:
            added_attributes_extended = pd.concat([added_attributes] * cleaned_results['val_results'].shape[0], ignore_index=True)
            Biomarker_Validation_Results_current =  pd.concat([added_attributes_extended,  cleaned_results['val_results']], axis = 1)
            Biomarker_Validation_Results = pd.concat([Biomarker_Validation_Results, Biomarker_Validation_Results_current], axis = 0)

        # Extend Biomarker_Validation_Results
        if cleaned_results['full_val_results'].shape[0] > 0:
            added_attributes_extended = pd.concat([added_attributes] * cleaned_results['full_val_results'].shape[0], ignore_index=True)
            Cross_Validation_and_Biomarker_Validation_Results_current =  pd.concat([added_attributes_extended,  cleaned_results['full_val_results']], axis = 1)
            Cross_Validation_and_Biomarker_Validation_Results = pd.concat([Cross_Validation_and_Biomarker_Validation_Results, Cross_Validation_and_Biomarker_Validation_Results_current], axis = 0)
            
        ranked_matrix = make_rank_matrix(cleaned_results["ranks"], fill_missing=None, one_based=True)

        for feature in ranked_matrix.index:
            Selected_Biomarker_Panels["Feature"].append(feature)
            Selected_Biomarker_Panels["Cohort"].append(dataset_name)
            Selected_Biomarker_Panels["OmicsLevel"].append(omics_levels_map[experiment_name])
            Selected_Biomarker_Panels["OmicsTypes"].append(omics_types_map[experiment_name])
            for featureSelector in featureSelectors:
                Selected_Biomarker_Panels[featureSelector].append(ranked_matrix.loc[feature, featureSelector])
                
Selected_Biomarker_Panels = pd.DataFrame(Selected_Biomarker_Panels)


WORKING ON ROSMAP DATASET
WORKING ON miRNA_data EXPERIMENT
WORKING ON dna_methylation_data EXPERIMENT
[run_pipeline] Skipping ML augmentation: cv_performance or val_results is empty.
WORKING ON gene_expression_data EXPERIMENT
WORKING ON miRNA_and_gene_expression_data EXPERIMENT
WORKING ON miRNA_and_dna_methylation_data EXPERIMENT
WORKING ON gene_expression_and_dna_methylation_data EXPERIMENT
WORKING ON miRNA_and_gene_expression_and_dna_methylation_data EXPERIMENT

WORKING ON MayoRNASeq DATASET
WORKING ON metabolomics_data EXPERIMENT
[run_pipeline] Skipping ML augmentation: cv_performance or val_results is empty.
WORKING ON gene_expression_data EXPERIMENT
WORKING ON proteomics_data EXPERIMENT
WORKING ON gene_expression_and_proteomics_data EXPERIMENT
WORKING ON metabolomics_and_gene_expression_data EXPERIMENT
WORKING ON metabolomics_and_proteomics_data EXPERIMENT
WORKING ON metabolomics_and_gene_expression_and_proteomics_data EXPERIMENT

WORKING ON BRCA DATASET
WORKING ON miRNA_data EXP

In [19]:
# Save Results
Cross_Validation_Results.to_csv(f"../BENCHMARKING/Cross_Validation_Results.csv", index = False)
Biomarker_Validation_Results.drop(columns = ["Unnamed: 0"], inplace = True)
Biomarker_Validation_Results.to_csv(f"../BENCHMARKING/Biomarker_Validation_Results.csv", index = False)
Cross_Validation_and_Biomarker_Validation_Results.drop(columns = ["Unnamed: 0"], inplace = True)
Cross_Validation_and_Biomarker_Validation_Results.to_csv(f"../BENCHMARKING/Cross_Validation_and_Biomarker_Validation_Results.csv", index = False)
Selected_Biomarker_Panels.to_csv(f"../BENCHMARKING/Selected_Biomarker_Panels.csv", index = False)